# l-1 trend filtering

В своей знаменитой статье [l-1 trend filtering](https://web.stanford.edu/~boyd/papers/l1_trend_filter.html) Стивен Бойд и его коллеги предложили подход к выявлению тренда в финансовых временных рядах с помощью выпуклой оптимизации. В частности, вместо построения стохастической модели временного ряда, они предложили отталкиваться от задачи - построить кусочно-линейную линию тренда, наилучшим образом аппроксимирующую данный временной ряд.

Это привело к задачи оптимизации следующего вида:

**TrendFilter problem**
$$
\min_{x \in R^T} \frac{1}{2}\|y - x\|_2^2 + \lambda \|Dx\|_1
$$
где $y \in R^T$ - временной ряд фактических цен акции (с момента времени $t = 1$ до момента времени $T$), матрица $D \in R^{(T-2) \times T}$ является трехдиагональной: $[Dx]_i = x_i - 2 \cdot x_{i+1} + x_{i+2} \quad$ ($1\leq i \leq T-2$).

Первый член в целевой функции отвечает за близость решения к фактическим данным, а второй форсирует решение  быть кусочно-линейной функцией с минимальным числом "изломов". 


In [1]:
import pandas as pd
import numpy as np

In [6]:
import pandas_datareader as pdr

In [2]:
import csv
data = pd.read_csv("data1.csv", sep = ";", encoding="utf-8", header = None, engine = 'python')

In [3]:
data.head()

,0,1,2,3,4,5,6,7,8
0,<TICKER>,<PER>,<DATE>,<TIME>,<OPEN>,<HIGH>,<LOW>,<CLOSE>,<VOL>
1,MOEX,D,20190228,000000,90.4000000,91.2200000,89.9100000,91.2200000,5820490
2,MOEX,D,20190301,000000,91.1000000,91.9500000,90.7200000,91.5700000,7745640
3,MOEX,D,20190304,000000,91.7100000,93.3700000,91.4400000,92.4400000,9692960
4,MOEX,D,20190305,000000,92.4300000,93.2200000,91.6000000,92.3800000,7908670


In [5]:
from pandas import DataFrame
data_frame = DataFrame(data)

In [6]:
y = np.array(data_frame[7])

In [7]:
y = np.delete(y, 0)

In [27]:
for i in range(y.size):
    y[i] = float(y[i])

2)Сделаем замену $Dx = z$, тогда получим эквивалентную задачу $$\min \frac{1}{2}\|y - x\|_2^2 + \lambda\|z\|_1$$ $$s.t~ Dx = z$$

Выпишем выражение для Лагранжиана

$$
L(x, z, \mu) = \frac{1}{2}\|y - x\|_2^2 + \lambda\|z\|_1 + \mu^T(Dx - z)
$$

Найдем инфимум Лагранжиана, для этого приравняем частные производные по $x$ и по $z$ к нулю.

$$\dfrac{\partial L}{\partial x} = -(y - x) + D^T\mu = 0 \Rightarrow y - x = D^T\mu$$


Заметим, что задача сепарабельна по $z$, тогда, разбив задачу на мелкие подзадачи, получим
$$\left\{
\begin{aligned}
&\lambda = \mu_i, z_i > 0
\\
&\lambda = -\mu_i, z_i < 0 
\\
&\mu \in [-\lambda; \lambda], z_i = 0
\end{aligned}
\right. \Rightarrow \lambda\|z\|_1 = \mu^Tz
$$


Подставим результат в выражение для Лагранжиана и получим двойственную задачу

$$
\begin{aligned}
&\min_{\mu} \frac{1}{2}\|D^T\mu\|_2^2 - \mu^TDy
\\
&s.t ~ -\lambda\mathbf{1}\leqslant \mu \leqslant \lambda\mathbf{1}
\end{aligned}
$$

3)Для решения прямой задачи решим двойственную, так как решение прямой выражается через решение двойственной следующим образом

$$
x^{opt} = y - D^T\mu^{opt}
$$
Решим эквивалентную задачу

$$
\begin{aligned}
\min_{\mu} \frac{1}{2}\|D^T\mu\|_2^2 - \mu^TDy + I_Q(\mu),
\end{aligned}
$$
где 
$$I_{Q}(\mu) = \left\{
\begin{aligned}
&0, ~-\lambda\mathbf{1}\leqslant \mu \leqslant \lambda\mathbf{1}
\\
&+\infty, ~otherwise
\end{aligned}
\right.
$$

Данная задача решается методом проксимального градиентного спуска, в котором итерация будет иметь следующий вид

$$
\mu_{k+1} = Prox_{\alpha_k h}(\mu_k - \alpha_k\nabla f(\mu_k)), ~h(\mu) = I_Q(\mu), ~f(\mu) = \frac{1}{2}\|D^T\mu\|_2^2 - \mu^TDy.
$$

$$
\nabla f(\mu) = DD^T\mu - Dy.
$$

В качестве начального приближения $\mu$ возьмем вектор из всех нулей, а за $\alpha_k$ возьмем обратное максимальное собственное число матрицы Гессе, в данном случае она равна $DD^T$.

In [382]:
m = np.zeros((y.size - 2))

In [371]:
D = np.zeros((y.size - 2, y.size))

In [374]:
A = D.dot(D.T)

In [375]:
l = 1/max(np.linalg.eigvals(A))

In [376]:
l

0.06250488437662041

In [372]:
for i in range(y.size - 2):
    D[i][i] = 1
    D[i][i+1] = -2
    D[i][i+2] = 1

In [295]:
def Prox(x):
    for i in range(x.size):
        if x[i] > 1:
            x[i] = 1
        else:
            if x[i] < -1:
                x[i] = -1
    return x

In [296]:
def Grad(x):
    return A.dot(x) - D.dot(y)

In [297]:
def MSE(x, y):
    s = 0.0
    for i in range(x.size):
        s+= ((x[i] - y[i])*(x[i] - y[i]))
    return s

In [298]:
q = 1e-04

In [383]:
m_k = np.zeros((y.size - 2))
m_k[0] = 1.0
sum = 0

In [384]:
while MSE(m, m_k) > q:
    m_k = m[:]
    m = Prox(m - l*Grad(m))
    sum +=1 

In [385]:
sum

202

In [302]:
x_0 = (y - D.T.dot(m))[:] 

In [305]:
1.0/2.0*MSE(y,x_0)+Norm_1(D.dot(x_0)) # Значение целевой функции в проксимальном методе

70.87518690522856

4) Положим параметр регуляризации равным единице $\lambda = 1$. Сделаем замену $Dx = z$, далее выпишем модифицированный Лагранжиан

$$
L_r(x, z, \mu) = \frac{1}{2}\|y - x\|_2^2 + \|z\|_1 + \frac{1}{2r}\|\mu + r(Dx - z)\|_2^2 - \dfrac{\|\mu\|^2_2}{2r}.
$$

Тогда схема ADMM будет выглядеть следующим образом

$$
\begin{aligned}
&x^{k+1} = argmin_{x}~ L_r(x, z^k, \mu^k)
\\
&z^{k+1} = argmin_{z}~ L_r(x^k, z, \mu^k)
\\
&\mu^{k+1} = \mu^k + r(Dx^{k+1} - z^{k+1})
\end{aligned}
$$

Сначала решим задачу минимизации Лагранжиана по $x$, взяв градиент, получим

$$
x^{k+1} = (I + rD^TD)^{-1}(rD^Tz^{k} - D^{T}\mu^k + y). 
$$

Далее по $z$. Заметим, что задача сепарабельна, поэтому 

$$
\begin{aligned}
&z_i^{k+1} = 1/r(\mu_i^k + r(Dx^{k})_i - 1),~ \mu_i^k + r(Dx^{k})_i > 1
\\
&z_i^{k+1} = 1/r(\mu_i^k + r(Dx^{k})_i + 1),~ \mu_i^k + r(Dx^{k})_i < -1
\\
&z_i^{k+1} = 1/r(\mu_i^k + r(Dx^{k})_i),~ \mu_i^k + r(Dx^{k})_i \in [-1; 1]
\end{aligned}
$$



In [306]:
r = 1000

In [307]:
C = D.T.dot(D)

In [308]:
B = r*C + np.eye(y.size)

In [309]:
x = np.zeros(y.size)
z = np.zeros(y.size - 2)
n = np.zeros(y.size - 2)

In [310]:
B_1 = np.linalg.inv(B)

In [311]:
def Step_z(z, s):
    for i in range(z.size):
        if s[i] > 1:
            z[i] = 1/r*(s[i] - 1)
        else:
            if s[i] < -1:
                z[i] = 1/r*(s[i] + 1)
            else:
                z[i] = 1/r*s[i]
    return z

In [312]:
def Step_x(z, n):
    return B_1.dot(r*D.T.dot(z) - D.T.dot(n) + y)

In [313]:
def Step_n(n, x, z):
    return n + r*(D.dot(x) - z)

In [314]:
x_k = np.zeros(y.size)
x_k[0] = 1.0

In [315]:
s = 0
while MSE(x_k, x) > q*10:
    x_k = x[:]
    x = Step_x(z, n)
    z = Step_z(z, n + r*D.dot(x))
    n = Step_n(n, x, z)
    s+=1

In [316]:
s

203

In [317]:
def Norm_1(x):
    s = 0
    for i in range(x.size):
        s += abs(x[i])
    return s

In [318]:
1.0/2.0*MSE(y,x)+Norm_1(D.dot(x)) # Значение целевой функции в ADMM

77.0957736314063

4)Сглаживание

$$f_{\mu} = \frac{1}{2}\|y-x\|_2^2 + \lambda\|Dx\|_1 - \frac{\mu}{2}\|u\|_2^2$$. 

Запишем в другом виде

$$f_{\mu} = \frac{1}{2}\|y-x\|_2^2+\max\limits_{-1\leq u_i \leq 1}\left[\sum_{i=1}^{n-2}\lambda (u_i(Dx)_i - \frac{\mu}{2}u_i^2\right].$$

$$L_i = \frac{\mu}{2}u_i^2 - \lambda u_i(Dx)_i + a_i(-1-u_i)+b_i(u_i-1)$$. 

Приравняв градиент нулю, выразим $u_i$:

$$u_i = \frac{a_i-b_i+\lambda(Dx)_i}{\mu},$$


Учитывая ограничения, получим

$$
u_i = \left\{\begin{aligned}
&-1,~\lambda(Dx)_i \leqslant -\mu
\\
&1,~\lambda(Dx)_i  \geqslant \mu
\\
&\dfrac{\lambda(Dx)_i}{\mu},~\lambda(Dx)_i \in (-\mu, \mu)
\end{aligned}\right.
$$


В силу дифференцируемости можем подставлять $u_{opt}$ в выражение для градиента


$$\nabla f_{\mu}(x) = x - y + \lambda D^Tu_{opt}.$$

Далее найдем константу Липшица

$d_2(u) \geqslant \frac{\sigma_2}{2}\|u\|_2^2$. $\sigma_2 = 1$, $\|D\|_{1,2} = \max \lambda_{DD^T}\Rightarrow L_{\mu} = \frac{\|D\|_{1,2}}{\mu} \Rightarrow L = L_{\mu} + L(1/2\|y -x\|_2^2) = L_{\mu} + 1$



$f_{\mu}(x_k) = \frac{1}{2}\|y-x_k\|_2^2 + \lambda<u_{opt}, Dx_k> - \frac{\mu\|u_{opt}\|_2^2}{2}, \nabla f_{\mu}(x_k) = x_k - y + \lambda D^Tu_{opt}.$

$$z_k = argmin_{z}\left[\nabla^T f_{\mu}(x_k)(z-x_k)+\frac{L}{2}\|z-x_k\|_2^2\right].$$

Приравняем градиент нулю, найдем $z$:

$$x_k - y + \lambda D^Tu_{opt} + L(z-x_k) = 0$$

$$z_k = -\frac{1}{L}(x_k-y+\lambda D^Tu_{opt}) + x_k$$

$$r_k = argmin_r\left[\frac{L}{\sigma_2}d_2(r) +  \sum\limits_{i=0}^k\alpha_i\left(f(x_i) + \nabla^Tf(x_i)(r-x_i)\right)\right].$$


$$r_k = argmin_r \left[\frac{L}{2}\|r\|_2^2 + \sum\limits_{i=0}^k\frac{i+1}{2}(f(x_i) + \nabla^Tf(x_i)(r-x_i))\right].$$

Приравняем градиент нулю, найдем $r$:

$$Lr + \sum\limits_{i=0}^k \frac{i+1}{2}\nabla f(x_i) = 0 \Rightarrow r_k = - \frac{1}{L}\sum\limits_{i=0}^k \frac{i+1}{2}\nabla f(x_i).$$

$$x_{k+1} = \frac{2}{k+3} r_k + \frac{k+1}{k+3}z_k.$$

In [355]:
m = 0.01
u = np.zeros(y.size - 2)
x = np.zeros(y.size)
z = np.zeros(y.size)
r = np.zeros(y.size)
x_k = np.zeros(y.size)
x_k[0] = 1
g = np.zeros(y.size)
L = max(np.linalg.eigvals(A))/m + 1
i = 0
lam = 1

In [347]:
def Smooth_optimal_u(u, x):
    for i in range(y.size - 2):
        if x[i] <= -m:
            u[i] = -1
        else:
            if x[i] >= m:
                u[i] = 1
            else:
                u[i] = x[i]/m
    return u

In [348]:
def Smooth_Grad(x, u):
    return x - y + lam*D.T.dot(u)

In [342]:
def Smooth_step_z(x, u):
    return -1/L*(x - y + lam*D.T.dot(u)) + x

In [356]:
while MSE(x_k, x) > q:
    x_k = x[:]
    u = Smooth_optimal_u(u, lam*D.dot(x))
    z = Smooth_step_z(x, u)
    g = Smooth_Grad(x, u)
    r = -float((k+1))*g/(2*L) + r
    x = 2/(i + 3)*r + (i + 1)/(i + 3)*z 
    i += 1

In [357]:
i

515

In [359]:
1.0/2.0*MSE(y,x)+Norm_1(D.dot(x)) # Значение целевой функции в сглаживании

71.23757717381176

Сравнение методов: 
1)Проксимальный градиентный спуск -- значение целевой функции $70.87$, количество итераций $202$

2)ADMM -- значение целевой функции $77.09$, количество итераций $203$

3)Сглаживание -- значение целевой функции $71.23$, количество итераций $505$.

# ADMM

Техника сглаживания оказала достаточно большое влияние на развитие методов восстановления тензоров (matrix completion problem является частным случаем). 

Рассмотрим задачу восстановления матрицы $Y$. Мы наблюдаем только небольшой набор элементов матрицы $Y$, множество индексов известных элементов обозначим через $E$. То есть мы знаем $Y_{ij}$ для всех $(i,j) \in E$, и не знаем $Y_{ij}$ для всех $(i,j) \notin E$.

Понятно, что без наличия какой-либо дополнительной информации о матрице $Y$ эта постановка является тривиальной: любая матрица $X$, для которой выполнено $X_{i,j} = Y_{i,j}, \quad (i,j) \in E$ является разумным ответом. Таких матриц бесконечно много. Соответственно задача восстановления матриц обычно рассматривается как частный случай задачи приближения матрицы, а критерий качества (целевая функция) говорит не столько о близости $X$ к $Y$, сколько о полезных свойствах матрицы $X$ (этим свойствам матрица $Y$ может и не обладать).

Одна из самых популярных постановок, с которой Вы наверняка сталкивались, например, строя рекомендательные системы, является задача о поиске наилучшего приближения заданной матрицы матрицей малого ранга:

$$
\begin{align*}
& \min_{X} rk(X) \\
& X_{i,j} = Y_{i,j}, \quad (i,j) \in E\\
\end{align*}
$$
Как известно, в общем случае эта задача является NP-трудной.

Для того, чтобы обойти это припятствие ранк матрицы аппроксимируется той или иной выпуклой функцией от матрицы $X$.

Опять же стандартным выбором является переход к постановке задачи с использованием 1-й нормы Шаттена (она же trace norm).

$\textbf{RegMC problem}$

$$
\begin{align*}
& \min_{X}\|X \|_* \\
& X_{i,j} = Y_{i,j}, \quad (i,j) \in E\\
\end{align*}
$$

Здесь $X_* = \sum \sigma_i(X)$. 







1) В силу эквивалентности первой нормы Шаттена для матриц с первой нормой для векторов, будет решать данную задачу аналогичным образом.
Построим эквивалентную задачу
$$
\begin{aligned}
&\min_X~ \|X\|_* + I_Q(Z)
\\
&s.t~ X - Z = 0.
\end{aligned}
$$

Здесь 
$$
I_Q(Z) = \left\{\begin{aligned}
&0,~ Z_{i,j} = Y_{i, j}, ~~(i,j) \in E
\\
&\infty, ~ otherwise
\end{aligned}
\right.
$$ 

В таком виде задача пригодна для решения методом ADMM.

2)Запишем Лагранжиан в следующем виде

$$
L(X, Z, U) = \|X\|_* + I_Q(Z) + \dfrac{1}{2r}\|U + r(X - Z)\|_F^2.
$$

Тогда 

$$
\begin{aligned}
&X^{k+1} = argmin_{X}\left(\|X\|_* + \dfrac{1}{2r}\|r(X - Z^{k}) + U^k\|_F^2\right)
\\
&Z^{k+1} = \Pi_Q\left(X^{k+1} + \dfrac{U^k}{r}\right)
\\
&U^{k+1} = U^k + r(X^{k+1} - Z^{k+1}).
\end{aligned}
$$

Найдем $argmin_{X}\left(\|X\|_* + \dfrac{1}{2r}\|r(X - Z^{k}) + U^k\|_F^2\right)$. Для более компактной записи положим $A = rZ^k - U^k$.

Под записью $\sigma(R)$ будет пониматься вектор, составленный из сингулярных чисел матрицы $R$. Далее заметим, что $\|R\|_* = \|\sigma(R)\|_1$, также пусть $R = QKW^T$ - сингулярное разложение матрицы, тогда

$$\langle QKW^T, QKW^T\rangle = \langle Q^TQKW^TW, K\rangle = \langle K, K\rangle = \|\sigma(R)\|_2^2.$$

Пусть теперь $A = WSV^T$ - сингулярное разложение. Тогда будем решать аналогичную задачу, но теперь для вектора $\sigma(A)$.

$$
argmin_{x}(\|x\|_{1} + \dfrac{1}{2r}\|rx - \sigma(A)\|_2^2)
$$

Заметим, что задача сепарабельная, тогда взяв производную покомпонентно, получим

$$
x_i = \left\{\begin{aligned}
&\dfrac{\sigma(A)_i - 1}{r}, ~\sigma(A)_i > 1
\\
&\dfrac{\sigma(A)_i + 1}{r}, ~\sigma(A)_i < -1
\\
&0, ~\sigma(A)_i \in [-1, 1]
\end{aligned}\right.
$$

Тогда $X^{k+1} = W\Sigma V^T$, где $\Sigma_{i,i} = x_i, \Sigma_{i,j} = 0~ \forall i \neq j$. 


Для нахождения $Z^{k+1}$ нужно взять проекцию на множество, а для этого достаточно положить $[X^{k+1} + U^k]_{i,j} = Y_{i,j}~~ \forall (i,j) \in E$.

In [2]:
import numpy as np

In [285]:
np.random.seed(0) #тестовая матрица для проверки алгоритма
Y = np.random.rand(100,100)

In [286]:
E = [[0,0],[0,1],[1,1],[1,3],[2,0],[2,2], [3,1], [3,3]] #набор известных индексов

In [287]:
for (i, j) in E: #значения в известных индексах
    Y[i][j] = 13
Y[0][2] = 12
Y[0][3] = 1
Y[3][0] = 18

In [295]:
n = Y.shape[0] #размеры матрицы
m = Y.shape[1]

In [296]:
np.linalg.norm(Y, 'nuc') #ядровая норма тестовой матрицы

348.13072273409006

In [297]:
def Projection_Q(Z): #Проекция на множество
    for [i, j] in E:
        Z[i][j] = Y[i][j]
    return Z

In [298]:
def Proximal_x(x, a): #Проксимальное отображение для вектора
    for i in range(n):
        if a[i] > 1:
            x[i] = (a[i] - 1)/r
        else: #отбросили один случай, так как сингулярные числа всегда положительны
            x[i] = 0
    return x

In [299]:
def Proximal_X(Z, U): #Проксимальное отображение для матрицы
    W, s, V = np.linalg.svd(r*Z - U, full_matrices=True)
    x = np.zeros(n)
    x = Proximal_x(x, s)
    S = np.zeros((n,m))
    for i in range(n): #меняем матрицу S с новыми сингулярными числами
        S[i][i] = x[i]
    return W.dot(S).dot(V)

In [300]:
def Step_U(U, X, Z): #шаг для матрицы U
    return U + r*X - r*Z

In [270]:
q = 0.001 #невязка

In [290]:
np.random.seed(0) #начальные приближения
X = np.random.rand(n,m)
Z = np.random.rand(n,m)
U = np.random.rand(n,m)
r = 10
a = 0
s = 0 

In [291]:
while abs(a - np.linalg.norm(Z, 'nuc')) > q:  #алгоритм работает, пока невязка не больше q
    X = Proximal_X(Z, U)
    a = np.linalg.norm(Z, 'nuc')
    Z = Projection_Q(X + U/r)
    U = Step_U(U, X, Z)
    s += 1

In [292]:
s #количество итераций

470

In [294]:
np.linalg.norm(Z, 'nuc')

56.196142261842475